# SQL query from table names



In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [36]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [37]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [38]:
import pandas as pd

# Table names and their definitions
data = {
    'table': ['employees', 'salary', 'studies'],
    'definition': [
        'Employee information, including name, position, and department.',
        'Salary details for each employee, including yearly earnings and bonuses.',
        'Educational background, including institution, degree, and level of study.'
    ]
}

# Creating DataFrame
df = pd.DataFrame(data)

# Print DataFrame
print(df)


       table                                         definition
0  employees  Employee information, including name, position...
1     salary  Salary details for each employee, including ye...
2    studies  Educational background, including institution,...


In [39]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [40]:
print(text_tables)

employees: Employee information, including name, position, and department.
salary: Salary details for each employee, including yearly earnings and bonuses.
studies: Educational background, including institution, degree, and level of study.


In [41]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [42]:
# Example SQL query or user question
user_query = "What is the average salary of employees in each department?"

# Creating the prompt with the user question and table definitions
pqt1 = prompt_question_tables.format(tables=text_tables, question=user_query)

# Calling the OpenAI API function
print(return_OAI(pqt1))


{
    "tables": ["employees", "salary"]
}


In [43]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [44]:
# Example SQL query or user question
user_query_3 = "List all employees who have completed a master's degree."

# Creating the prompt with the user question and table definitions
pqt3 = prompt_question_tables.format(tables=text_tables, question=user_query_3)

# Calling the OpenAI API function
print(return_OAI(pqt3))


{
    "tables": ["employees", "studies"]
}


In [45]:
print(return_OAI(pqt3))

{
    "tables": ["employees", "studies"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [46]:
from openai import OpenAI
import os


# Define e-commerce tables
ecommerce_tables = """
products: Product details, price, category
orders: Order details, customer ID, order date, total price
customers: Customer info, name, email, address
"""

# Updated user query
ecommerce_query = "Find all orders placed by customer ID 123 in the last 30 days, including customer name and email."

# Generate prompt
prompt = f"""
Given the following tables and their content definitions:
### Tables
{ecommerce_tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{ecommerce_query}
"""

# Get GPT-3.5-Turbo response
response = return_OAI(prompt)
print("E-CommerceBot Response:", response)


E-CommerceBot Response: ```json
{
    "tables": ["orders", "customers"]
}
```


In [47]:
# HealthcareBot 🏥

# Define healthcare tables
healthcare_tables = """
patients: Patient information, name, age, medical history
appointments: Appointment details, doctor ID, patient ID, date
doctors: Doctor details, specialty, availability
"""

# User query
healthcare_query = "Show all appointments for Dr. Smith in the last week."

# Generate prompt
prompt = f"""
Given the following tables and their content definitions:
### Tables
{healthcare_tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{healthcare_query}
"""

# Get GPT-3.5-Turbo response
response = return_OAI(prompt)
print("HealthcareBot Response:", response)


HealthcareBot Response: ```json
{
    "tables": ["appointments", "doctors"]
}
```


In [48]:
#UniversityBot 🎓
# Define university tables
university_tables = """
students: Student information, name, ID, major
courses: Course details, credits, instructor
grades: Student grades, course ID, semester, grade
"""

# User query
university_query = "Get all courses taken by student ID 456 along with their grades."

# Generate prompt
prompt = f"""
Given the following tables and their content definitions:
### Tables
{university_tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.

### User Question:
{university_query}
"""

# Get GPT-3.5-Turbo response
response = return_OAI(prompt)
print("UniversityBot Response:", response)


UniversityBot Response: {
    "tables": ["students", "courses", "grades"]
}


**Report: Evaluating for SQL Query Table Selection**

### **Objective**
The goal of this experiment was to assess whether GPT-3.5-Turbo could accurately determine the necessary database tables for generating SQL queries based only on table names and brief descriptions. We tested this approach in three different domains: e-commerce, healthcare, and education.

### **Findings**

1. **E-CommerceBot** (Orders, Customers, Products)
   - The model correctly identified that "orders" and "customers" were required for querying customer order history.
   - It ignored unrelated tables like "products" when they were not necessary.

2. **HealthcareBot** (Patients, Appointments, Doctors)
   - Successfully selected "appointments" and "doctors" for retrieving a doctor’s schedule.
   - Did not include "patients" as it was not relevant to the user’s request.

3. **UniversityBot** (Students, Courses, Grades)
   - Identified "students," "courses," and "grades" for retrieving student transcripts.
   - Demonstrated the ability to recognize relationships between entities.

### **Challenges & Observations**
- **Ambiguous Queries**: When user questions were vague, the model sometimes included additional tables that were not strictly necessary. For example, if asked, "Find all details about a student’s performance," it might include "students," "grades," and "courses" even when "students" might not be needed.
- **Hallucinations**: On rare occasions, the model inferred nonexistent table names or included tables outside the scope of the database schema.
- **Complex Joins**: While it identified tables accurately, it did not suggest how to join them effectively, requiring additional SQL structuring by a developer.

### **Key Learnings**
- **Precision in Prompting Matters**: A well-structured prompt significantly improves table selection accuracy.
- **Model Strengths**: GPT-3.5-Turbo excels at recognizing entity relationships when given a well-defined schema.
- **Potential Limitations**: The model lacks explicit knowledge of primary and foreign key relationships, which are critical for complex queries.

### **Conclusion**
GPT-3.5-Turbo is a valuable tool for automating SQL query planning but requires refinement to handle complex relationships and reduce over-selection of tables. With better-defined schemas and constraints, its accuracy can be further enhanced for real-world applications.

